```{contents}
```
## Model Provider

In **LangChain**, a **model provider** is:

> A backend service that hosts an LLM, exposed through a **standardized LangChain interface** (`ChatModel`).

LangChain **decouples your code from the provider**.

```
Your App
  ↓
LangChain ChatModel abstraction
  ↓
Provider SDK / API
  ↓
Actual LLM
```

---

### Why Model Providers Are Abstracted

Without LangChain:

* You write **provider-specific SDK code**
* Hard to switch models
* Tool calling, streaming, retries differ

With LangChain:

* Same API
* Same prompt format
* Same agent code
* Swap providers with **1 line change**

---

### Common Model Providers in LangChain

| Provider                | LangChain Class          | Typical Models      |
| ----------------------- | ------------------------ | ------------------- |
| OpenAI                  | `ChatOpenAI`             | gpt-4o, gpt-4o-mini |
| Anthropic               | `ChatAnthropic`          | claude-3-sonnet     |
| Google                  | `ChatGoogleGenerativeAI` | gemini-1.5          |
| Meta (local via Ollama) | `ChatOllama`             | llama3              |
| Mistral AI              | `ChatMistralAI`          | mistral-large       |
| Hugging Face            | `ChatHuggingFace`        | open-source models  |

---

### Provider-Agnostic Rule (Key Concept)

> **LangChain code should not care which provider is used.**

This should work **unchanged**:

```python
prompt | chat_model
```

---

### Demonstration: OpenAI Provider

-  Install

```bash
pip install langchain-openai
```

- Code



In [ ]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2
)

response = llm.invoke("Explain RAG in one sentence")
print(response.content)




**Characteristics**

* Best tool calling
* Strong reasoning
* Paid API

---

### Demonstration: Anthropic (Claude)

#### Install

```bash
pip install langchain-anthropic
```

#### Code



In [ ]:

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0.2
)

response = llm.invoke("Explain RAG in one sentence")
print(response.content)




**Characteristics**

* Long context
* Strong summarization
* Conservative responses

---

### Demonstration: Google Gemini

#### Install

```bash
pip install langchain-google-genai
```

#### Code



In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.2
)

response = llm.invoke("Explain RAG in one sentence")
print(response.content)




**Characteristics**

* Multimodal
* Competitive latency
* Google ecosystem

---

### Demonstration: Local Models via Ollama (No Cloud)

#### Install

```bash
pip install langchain-community
```

#### Prerequisite

```bash
ollama pull llama3
```

#### Code



In [ ]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(
    model="llama3",
    temperature=0.2,
    base_url="http://localhost:11435"
)

response = llm.invoke("Explain RAG in one sentence")
print(response.content)




**Characteristics**

* Fully local
* No data leaves machine
* Slower than cloud

---

### Same Prompt, Different Providers (Power of Abstraction)



In [ ]:

def ask(llm):
    return llm.invoke("What is LangChain?").content

ask(ChatOpenAI(model="gpt-4o-mini"))
ask(ChatAnthropic(model="claude-3-sonnet-20240229"))
ask(ChatOllama(model="llama3"))




**No code rewrite.**

---

### Tool Calling Works Across Providers



In [9]:
from langchain_openai import ChatOpenAI
def ticket_count(source: str) -> int:
    return 120

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools([ticket_count])

llm_with_tools.invoke("How many tickets are there in Jira?")


AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 45, 'total_tokens': 59, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cov9Vk9jNkEmHA4bJv0B36TIQIcpr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b3cd4-a81d-7092-9a9a-3364e5ebc284-0', tool_calls=[{'name': 'ticket_count', 'args': {'source': 'jira'}, 'id': 'call_7Apqfue4oaVGeUQogIPuZrx0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 45, 'output_tokens': 14, 'total_tokens': 59, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})



Same pattern works for:

* OpenAI
* Anthropic
* Gemini
  (if provider supports tool calling)

---

### Streaming Is Provider-Independent

```python
for chunk in llm.stream("Explain LangChain"):
    print(chunk.content, end="")
```

---

### How to Choose a Provider (Production View)

| Requirement       | Recommended Provider |
| ----------------- | -------------------- |
| Lowest latency    | OpenAI               |
| Best tool calling | OpenAI               |
| Longest context   | Anthropic            |
| Multimodal        | Google               |
| Offline / private | Ollama               |
| Lowest cost       | Mix via routing      |

---

### Advanced Pattern: Provider Routing


```python
if task == "summarization":
    llm = ChatAnthropic(...)
else:
    llm = ChatOpenAI(...)
```

LangChain supports **dynamic routing** without changing chains.

---

### Interview-Ready Summary

> “In LangChain, model providers are abstracted behind ChatModel interfaces. This allows us to switch between OpenAI, Anthropic, Google, or local models without changing prompts, agents, or RAG pipelines.”

---

### One Rule to Remember

> **Never hard-code provider logic into your business code.
> Always depend on LangChain’s model abstraction.**